<a href="https://colab.research.google.com/github/fionabay/slob/blob/main/SVI_cudf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Aug  6 02:52:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import cudf
%load_ext cudf.pandas

sample calibartion of implied volatility surface

In [3]:
%%cudf.pandas.profile

import numpy as np
from scipy import optimize
from matplotlib import pyplot as plt

spot = 1.3444
forward = 1.342782
t = 30 / 365.0
vols = np.array([9.420, 9.772, 9.237, 10.144, 9.196, 10.724, 9.265, 11.161, 9.390, 11.908, 9.751]) / 100
strikes = np.array([1.34342148, 1.35800697, 1.32950654, 1.37006384, 1.31948358, 1.38700437, 1.30670715,1.39978993, 1.29765089, 1.42124726, 1.28287975])
total_implied_variance = np.power(t * vols,2)

def svinatural(k, param):
  # The natural SVI parameterization
    delta = param[0];
    omega = param[1];
    epsilon = param[2];
    rho = param[3];
    miu= param[4];

    estimate = delta + 0.5*omega * (1+ epsilon* rho *(k -miu)+np.sqrt( (epsilon*(k-miu)+rho))**2 +(1-rho*rho))
    return estimate

def targetfunction(x,total_implied_variance):
    value=0
    for i in range(len(total_implied_variance)):
        model_total_implied_variance = svinatural(np.log(strikes[i] / forward), x);
        value =value+np.power(total_implied_variance[i]  - model_total_implied_variance,2);
    return value**0.5

bounds = [(-1000, 1000),(0, 1000),(0.1, 1000),(-0.9999, 0.9999),(-10000, 1)]
x0=[0.1,0.2,1,0.0001,-99]
result = optimize.minimize(targetfunction,x0=x0,args=(total_implied_variance),bounds=bounds)
x=result.x
print(x)



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/cudf/pandas/profiler.py", line 97, in __enter__
    sys.settrace(self._tracefunc)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/cudf/pandas/profiler.py", line 116, in __exit__
    sys.settrace(self._oldtrace)



[ 5.15830327e-05  2.48588708e-02  1.00002244e-01 -9.99897506e-01
 -9.89435351e+01]


                                                                                              
                                  Total time elapsed: 1.136 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘